# 03 – Inventory Optimization (EOQ/ROP/Safety Stock)
**Author:** Mousumi Paul | Feb 2025

## 1. Setup

In [ ]:
import sys
sys.path.append('../src')
import pandas as pd
from inventory_optimization import (
    run_inventory_optimization, eoq, safety_stock, reorder_point,
    plot_eoq_cost_curve, plot_stock_health, plot_cost_breakdown
)
print('✅ Ready')

## 2. Load Parameters

In [ ]:
params = pd.read_csv('../data/raw/inventory_params.csv')
params

## 3. Run Optimization

In [ ]:
inv_df = run_inventory_optimization(params, ordering_cost=2500, z_score=1.65,
                                     lead_time_days=14, working_days=250)
inv_df[['Category','EOQ_Units','Safety_Stock_Units','Reorder_Point_Units',
        'Days_of_Supply','Total_Inventory_Cost_INR','Alert_Status']]

## 4. Full Results Table

In [ ]:
inv_df.to_csv('../data/processed/inventory_optimization_output.csv', index=False)
inv_df

## 5. Stock Health Dashboard Chart

In [ ]:
plot_stock_health(inv_df, save_path='../outputs/charts/stock_health.png')

## 6. Cost Breakdown Chart

In [ ]:
plot_cost_breakdown(inv_df, save_path='../outputs/charts/cost_breakdown.png')

## 7. EOQ Cost Curve – Electronics

In [ ]:
row = params[params['Category']=='Electronics'].iloc[0]
hc_pu = row['Unit_Cost_INR'] * row['Holding_Cost_Pct']
plot_eoq_cost_curve('Electronics', row['Annual_Demand_Units'], 2500, hc_pu,
                    save_path='../outputs/charts/eoq_curve_electronics.png')

## 8. Cost Savings Analysis (Pre vs Post EOQ)

In [ ]:
# Simulate 'before' scenario: fixed large order qty = 1000 units
import numpy as np, math
from inventory_optimization import annual_holding_cost, annual_ordering_cost

before_rows = []
after_rows  = []
for _, row in params.iterrows():
    hc_pu    = row['Unit_Cost_INR'] * row['Holding_Cost_Pct']
    opt_eoq  = eoq(row['Annual_Demand_Units'], 2500, hc_pu)
    ss       = safety_stock(row['Demand_StdDev'], 14)
    fixed_qty = 1000  # unoptimized fixed order

    ahc_before = annual_holding_cost(fixed_qty, ss, hc_pu)
    aoc_before = annual_ordering_cost(row['Annual_Demand_Units'], fixed_qty, 2500)
    ahc_after  = annual_holding_cost(opt_eoq, ss, hc_pu)
    aoc_after  = annual_ordering_cost(row['Annual_Demand_Units'], opt_eoq, 2500)

    saving     = (ahc_before + aoc_before) - (ahc_after + aoc_after)
    before_rows.append(ahc_before + aoc_before)
    after_rows.append(ahc_after  + aoc_after)

cats = params['Category'].tolist()
savings_pct = [(b-a)/b*100 for b,a in zip(before_rows, after_rows)]
print('\n💰 Cost Savings by Category:')
for c, b, a, s in zip(cats, before_rows, after_rows, savings_pct):
    print(f'  {c:<25} Before: ₹{b:>10,.0f}  After: ₹{a:>10,.0f}  Saving: {s:.1f}%')
total_before = sum(before_rows)
total_after  = sum(after_rows)
total_saving_pct = (total_before-total_after)/total_before*100
print(f'  {"TOTAL":<25} Before: ₹{total_before:>10,.0f}  After: ₹{total_after:>10,.0f}  Saving: {total_saving_pct:.1f}%')